# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn import ensemble, cross_validation, learning_curve, metrics, datasets

import numpy as np
%pylab inline

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
import xgboost as xgb

In [3]:
boston_ds = datasets.load_boston()

In [5]:
boston_ds.keys()

['data', 'feature_names', 'DESCR', 'target']

In [100]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(boston_ds.data, boston_ds.target, 
                                                                                     test_size = 0.25)

In [147]:
train_data = boston_ds.data [:380]
test_data = boston_ds.data [-126:]
train_labels = boston_ds.target [:380]
test_labels = boston_ds.target [-126:]

(506, 13)

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

L' = 2 * (y - a(x))

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [18]:
from sklearn import tree

In [148]:
DecTreeRegress_mass = np.array ([])
coef_mass = np.array ([0.9]*50)
vect = train_labels


def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, 
                 coeff in zip(DecTreeRegress_mass, coef_mass)]) for x in X]

for i in range (0,50,1):
    dec_tree_regress = tree.DecisionTreeRegressor (max_depth=5,random_state=42)
    dec_tree_regress.fit (train_data, vect)
    DecTreeRegress_mass = np.append (DecTreeRegress_mass, [dec_tree_regress], axis = 0)
    z = gbm_predict(train_data)
    vect = train_labels - z

In [149]:
predicts = gbm_predict (test_data)
RMSE = metrics.mean_squared_error (test_labels, predicts)**0.5
print RMSE

5.455472074527535


In [150]:
answr1 = open ('answer1.txt', 'w')
answr1.write (str(RMSE))
answr1.close()

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [151]:
DecTreeRegress_mass1 = np.array([])
coef_mass2 = np.array([])
for i in range (0,50,1):
    coef_mass2 = np.append (coef_mass2, 0.9 / (1.0 + i))

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, 
                 coeff in zip(DecTreeRegress_mass1, coef_mass2)]) for x in X]

for i in range (0,50,1):
    dec_tree_regress = tree.DecisionTreeRegressor (max_depth=5,random_state=42)
    dec_tree_regress.fit (train_data, vect)
    DecTreeRegress_mass1 = np.append (DecTreeRegress_mass1, [dec_tree_regress], axis = 0)
    z = gbm_predict(train_data)
    vect = train_labels - z


In [152]:
predicts = gbm_predict (test_data)
RMSE = metrics.mean_squared_error (test_labels, predicts)**0.5
print RMSE

4.709471186581687


In [153]:
answer2 = open ('answer2', 'w')
answer2.write (str(RMSE))
answer2.close()

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [154]:
answer3 = open ('answer3', 'w')
answer3.write ('2 3')
answer3.close()

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [129]:
from sklearn import linear_model

In [155]:
reggres = linear_model.LinearRegression ()
reggres.fit (train_data, train_labels)
zr = reggres.predict (test_data)
RMSE = metrics.mean_squared_error (test_labels, zr)**0.5
print RMSE

7.848121796479934


In [156]:
answer4 = open ('answer4', 'w')
answer4.write (str(RMSE))
answer4.close()